# Confidence Interval Construction

## Using True Value of $\sigma^2$

In [ ]:
import numpy as np

np.random.seed(130828)   # for reproducibility (optional)
a = 0.5
n = 500
num_sims = 1000
z_alpha = 1.96  # 97.5th percentile of standard normal for a 95% CI

# True asymptotic variance for sample mean of AR(1)
sigma_sq = (1/(1 - a**2)) * ((1 + a) / (1 - a))  

coverage_count = 0

for _ in range(num_sims):
    # Generate Z_1,...,Z_n
    Z = np.zeros(n)
    Z[0] = np.random.normal(loc=0, scale=np.sqrt(1/(1 - a**2)))
    for t in range(1, n):
        eps = np.random.normal(loc=0, scale=1.0)
        Z[t] = a * Z[t - 1] + eps
    
    # Compute the sample mean of Z
    Z_bar = np.mean(Z)
    
    # Construct the 95% CI using known theoretical (asymptotic) variance
    se = np.sqrt(sigma_sq / n) #The standard error for Z_bar is sqrt(sigma_sq / n)
    lower = Z_bar - z_alpha * se
    upper = Z_bar + z_alpha * se
    
    # Check coverage of the true mean (which is 0)
    if lower <= 0 <= upper:
        coverage_count += 1

coverage_proportion = coverage_count / num_sims

print(f"Empirical coverage probability (Method 1) = {coverage_proportion:.3f}")


Empirical coverage probability (Method 1) = 0.956


## Using Sample Variance

In [ ]:
np.random.seed(130828)
a = 0.5
n = 500
num_sims = 1000
z_alpha = 1.96  # 97.5th percentile of the standard normal

coverage_count = 0

for _ in range(num_sims):
    # Generate Z_1,...,Z_n
    Z = np.zeros(n)
    Z[0] = np.random.normal(loc=0, scale=np.sqrt(1/(1 - a**2)))
    for t in range(1, n):
        eps = np.random.normal(loc=0, scale=1.0)
        Z[t] = a * Z[t - 1] + eps

    # Sample mean
    Z_bar = np.mean(Z)

    # Naive sample variance (ignoring correlation)
    s2 = np.var(Z, ddof=1)  # ddof=1 for unbiased sample variance

    # Construct the naive 95% CI
    se_naive = np.sqrt(s2 / n)  
    lower = Z_bar - z_alpha * se_naive
    upper = Z_bar + z_alpha * se_naive

    # 5. Check if the true mean (0) is within this CI
    if lower <= 0 <= upper:
        coverage_count += 1

coverage_proportion = coverage_count / num_sims
print(f"Empirical coverage probability (Method 2, naive sample variance) = {coverage_proportion:.3f}")


Empirical coverage probability (Method 2, naive sample variance) = 0.738


## Using Sectioning Method

In [ ]:
np.random.seed(130828)
a = 0.5
n = 500
num_sims = 1000
z_alpha = 1.96  # for ~95% CI

# Choose how to section the data
L = 10           # number of batches
B = n // L       # length of each batch (should be 50 if n=500 and L=10)

coverage_count = 0

for _ in range(num_sims):
    # Generate AR(1) data
    Z = np.zeros(n)
    Z[0] = np.random.normal(loc=0, scale=np.sqrt(1/(1 - a**2)))
    for t in range(1, n):
        eps = np.random.normal(loc=0, scale=1.0)
        Z[t] = a * Z[t - 1] + eps

    # Partition Z into L batches of length B, compute each batch mean
    
    M = []                  # M_i = mean of batch i
    for i in range(L):
        start_idx = i * B
        end_idx = (i + 1) * B
        M_i = np.mean(Z[start_idx:end_idx])
        M.append(M_i)
    M = np.array(M)

    # Overall sample mean: same as average of M_i
    Z_bar = np.mean(Z)
    
    # Batch-means formula: 
    #   var(Z_bar) = sample_var_of_M / L,
    #   sample_var_of_M = sum((M_i - Z_bar)^2) / (L - 1)

    M_var = np.sum((M - Z_bar)**2) / (L - 1)
    var_Z_bar = M_var / L  # batch-means estimate of var(Z_bar)
    se_sectioning = np.sqrt(var_Z_bar)

    # Construct the 95% CI
    lower = Z_bar - z_alpha * se_sectioning
    upper = Z_bar + z_alpha * se_sectioning

    # Check coverage
    if lower <= 0 <= upper:
        coverage_count += 1

coverage_proportion = coverage_count / num_sims
print(f"Empirical coverage probability (Method 3, sectioning) = {coverage_proportion:.3f}")


Empirical coverage probability (Method 3, sectioning) = 0.918
